In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [2]:
#Define spache-time manifold and coordinates

n = 3
M = Manifold(1+n, 'M', structure='Lorentzian')
X.<t,r,th,ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi")
coords = (t, r, th, ph)

In [3]:
eps = var('eps')

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)
omega = function('omega')(r,th)

nu = function('nu')(r)
lamb = function('lamb')(r)

Omegae = function('Omegaevalue')(eps)
Ne = function('Ne')(r,eps)

In [4]:
def cosmotensors(gedd):
    
    geuu = gedd.inverse()

    chris =  M.tensor_field(1,2,'chris')
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                chris[i,j,k] = 0.5*sum(geuu[i,dd]*(diff(gdd[dd,j],coords[k])
                                                  + diff(gdd[dd,k],coords[j])
                                                  - diff(gdd[j,k],coords[dd])) for dd in range(0,4))

    riem =  M.tensor_field(1,3,'riem')
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                for l in range(0,4):
                    riem[i,j,k,l] = (diff(chris[i,j,l],coords[k])
                                      - diff(chris[i,j,k],coords[l])
                                      + sum(chris[i,k,dd]*chris[dd,l,j]
                                          - chris[i,l,dd]*chris[dd,k,j] for dd in range(0,4)))

    ricci = M.tensor_field(0,2,'ricci')
    for i in range(0,4):
        for k in range(0,4):
            ricci[i,k] = sum(riem[dd,i,dd,k] for dd in range(0,4))

    s_curv = geuu['^{ab}']*ricci['_ab']
    
    eins = M.tensor_field(0,2,'eins')
    eins = ricci - gdd*s_curv/2   
    
    return {'inverse_metric': geuu,
            'christoffel': chris,
            'riemann': riem,
            'ricci': ricci,
            'curvature': s_curv,
            'einstein': eins}

In [5]:
gdd = M.tensor_field(0,2,'gdd')
gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

K1 = M.tensor_field(0,2,'K1')
K1[0,3] = omega*r^2*sin(th)^2
K1[3,0] = omega*r^2*sin(th)^2

K2 = M.tensor_field(0,2,'K2')
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = M.metric()
for i in range(0,n+1):
    for j in range(0,n+1):
        gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]

tensors = cosmotensors(gedd)
geuu = tensors['inverse_metric']
ricci = tensors['ricci']
s_curv = tensors['curvature']
eins = tensors['einstein']                                                                             #G_\mu\nu e

deins = M.tensor_field(0,2,'deins')
ddeins = M.tensor_field(0,2,'ddeins')
for i in range(0,n+1):
    for j in range(0,n+1):
        deins[i,j] = diff(eins[i,j].expr(),eps)
        ddeins[i,j] = diff(eins[i,j].expr(),eps,eps)

Tengo métrica
Tengo chris
Tengo riemann
Tengo ricci
Tengo eins
Tengo tó


In [6]:
show("ge0 = ", gdd[:])
show("K1 = ", K1[:])
show("K2 = ", K2[:])
show("ge = ", gedd[:])

'ge0 = ' [     -e^nu(r)             0             0             0]
[            0     e^lamb(r)             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

'K1 = ' [                         0                          0                          0 r^2*omega(r, th)*sin(th)^2]
[                         0                          0                          0                          0]
[                         0                          0                          0                          0]
[r^2*omega(r, th)*sin(th)^2                          0                          0                          0]

'K2 = ' [2*r^2*omega(r, th)^2*sin(th)^2 - 4*e^nu(r)*h(r, th)                                                   0                                                   0                                                   0]
[                                                  0                                4*e^lamb(r)*m(r, th)                                                   0                                                   0]
[                                                  0                                                   0                                      4*r^2*k(r, th)                                                   0]
[                                                  0                                                   0                                                   0                            4*r^2*k(r, th)*sin(th)^2]

'ge = ' [eps^2*r^2*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - e^nu(r)                                                                       0                                                                       0                                          eps*r^2*omega(r, th)*sin(th)^2]
[                                                                      0                                  2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r)                                                                       0                                                                       0]
[                                                                      0                                                                       0                                              2*eps^2*r^2*k(r, th) + r^2                                                                       0]
[                                         eps*r^2*omega(r, th)*sin(th)^2                                                                       0                                                                       0                                  (2*eps^2*r^2*k(r, th) + r^2)*sin(th)^2]

In [7]:
#4-velocities of the fluid for any epsilon

ue = M.vector_field("ue")
ue[0] = Ne; ue[1] = 0; ue[2] = 0; ue[3] = Ne*Omegae

u = M.vector_field("u")
u[0] = exp(-nu/2); u[1] =  0; u[2] =  0; u[3] =  0

Nevalue = solve(u[0].expr()==ue[0].expr(), Ne)[0].rhs()                                                    #Eq 4.20
Omegaevalue = solve(u[3].expr()==ue[3].expr(), Omegae)[0].rhs()                                            #Eq 4.20

In [10]:
show("Ne0 = ", Nevalue)
show("Omegae0 = ", Omegaevalue)

'Ne0 = ' e^(-1/2*nu(r))

'Omegae0 = ' 0

In [9]:
u1 = M.vector_field("u1")
u2 = M.vector_field("u2")

u10 = function('u10')(r)
u20 = function('u20')(r)

Omega1 = function('Omega1')(eps)
Omega2 = function('Omega2')(eps)

s0 = eps == 0
s1 = Ne.subs(eps==0) == Nevalue
s2 = Omegae.subs(eps==0) == Omegaevalue
s3 = diff(Ne,eps).subs(eps==0) == u10
s4 = diff(Omegae,eps).subs(eps==0) == Omega1
s5 = diff(Ne,eps,eps).subs(eps==0) == u20
s6 = diff(Omegae,eps,eps).subs(eps==0) == Omega2

sustituciones = (s0,s1,s2,s3,s4,s5,s6)

for i in range(0,n+1):
    u1[i] = diff(ue[i].expr(),eps).subs(sustituciones)
    u2[i] = diff(ue[i].expr(),eps,eps).subs(sustituciones)

Ue = ue.down(gedd)
geUeue = Ue['_a']*ue['^a']
dgeUeue = diff(geUeue.expr(), eps).subs(sustituciones) == 0
u10value = solve(dgeUeue, u10)[0].rhs()
s7 = u10 == u10value
u1.apply_map(lambda cmp: cmp.subs(u10==u10value))                                                          #Eq 4.21

ddgeUeue = diff(geUeue.expr(), eps, eps).subs(sustituciones) == 0
u20value = solve(ddgeUeue, u20)[0].rhs()
s8 = u20 == u20value
u2.apply_map(lambda cmp: cmp.subs(u20==u20value).subs(u10==u10value))                                      #Eq 4.22

In [11]:
show(u1.display())
show(u2.display())

u1 = Omega1(eps)*e^(-1/2*nu(r)) ∂/∂ph

u2 = ((r^2*Omega1(eps)^2 + 2*r^2*Omega1(eps)*omega(r, th) + r^2*omega(r, th)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-3/2*nu(r)) ∂/∂t + Omega2(eps)*e^(-1/2*nu(r)) ∂/∂ph

In [12]:
deins.apply_map(lambda cmp: cmp.subs(sustituciones))                                             #d/de(G_\mu\nu)|e=0
ddeins.apply_map(lambda cmp: cmp.subs(sustituciones))                                        #d^2/de^2(G_\mu\nu)|e=0

In [13]:
Ee = function('Ee')(r,eps)
Pe = function('Pe')(r,eps)

eins0 = M.tensor_field(0,2, 'eins0')
Tedd = M.tensor_field(0,2, 'Tedd')
Tdd = M.tensor_field(0,2, 'Tdd')
for i in range(0,n+1):
    for j in range(0,n+1):
        eins0[i,j] = eins[i,j]
        Tedd[i,j] = (Ee + Pe)*Ue[i]*Ue[j] + Pe*gedd[i,j]                                                 #T_\mu\nu e
        Tdd[i,j] = Tedd[i,j]

eins0.apply_map(lambda cmp: cmp.subs(sustituciones))                                                       #G_\mu\nu
Tdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                                         #T_\mu\nu

In [25]:
show("Te0 =", Tdd[:])

'Te0 =' [      E(r)*e^nu(r)                  0                  0                  0]
[                 0     P(r)*e^lamb(r)                  0                  0]
[                 0                  0           r^2*P(r)                  0]
[                 0                  0                  0 r^2*P(r)*sin(th)^2]

In [15]:
kappa = 8*pi
E = function('E')(r)
P = function('P')(r)

s9 = Ee.subs(eps==0) == E
s10 = Pe.subs(eps==0) == P
sustituciones = (s1,s2,s3,s4,s5,s6,s7,s8,s9,s10)
Tdd.apply_map(lambda cmp: cmp.subs(sustituciones))
E_tt = solve(eins0[0,0].expr()==kappa*Tdd[0,0].expr(),E)[0].rhs()                                            #Eq 3.7
P_rr = solve(eins0[1,1].expr()==kappa*Tdd[1,1].expr(),P)[0].rhs()                                            #Eq 3.8
P_thth = solve(eins0[2,2].expr()==kappa*Tdd[2,2].expr(),P)[0].rhs()
P_phph = solve(eins0[3,3].expr()==kappa*Tdd[3,3].expr(),P)[0].rhs()
ddnu = solve(P_rr == P_thth, diff(nu,r,r))[0].rhs()                                                          #Eq 3.9

In [19]:
show("E = ", E_tt)
show("Prr = ", P_rr)
show("ddnu =", ddnu)

'E = ' 1/8*(r*diff(lamb(r), r) + e^lamb(r) - 1)*e^(-lamb(r))/(pi*r^2)

'Prr = ' 1/8*(r*diff(nu(r), r) - e^lamb(r) + 1)*e^(-lamb(r))/(pi*r^2)

'ddnu =' -1/2*(r^2*diff(nu(r), r)^2 - 2*r*diff(lamb(r), r) - (r^2*diff(lamb(r), r) + 2*r)*diff(nu(r), r) + 4*e^lamb(r) - 4)/r^2

In [18]:
dlamb = solve(E==E_tt,diff(lamb,r))[0].rhs()                                                                 #Eq 3.10
dnu = solve(P==P_rr,diff(nu,r))[0].rhs()                                                                     #Eq 3.11

In [20]:
show(LatexExpr("dlamb = "), dlamb)
show(LatexExpr("dnu = "), dnu)

dlamb =  (8*pi*r^2*E(r)*e^lamb(r) - e^lamb(r) + 1)/r

dnu =  (8*pi*r^2*P(r)*e^lamb(r) + e^lamb(r) - 1)/r

In [21]:
h0 = function('h0')(r,th)
k0 = function('k0')(r,th)
m0 = function('m0')(r,th)

h2 = function('h2')(r,th)
k2 = function('k2')(r,th)
m2 = function('m2')(r,th)

m = m0 + m2*(3*cos(th)^2-1)/2
k = k0 + k2*(3*cos(th)^2-1)/2
h = h0 + h2*(3*cos(th)^2-1)/2

In [29]:
E1 = function('E1')(r)
E2 = function('E2')(r)
P1 = function('P1')(r)
P2 = function('P2')(r)

s11 = diff(Ee,eps).subs(eps==0) == E1
s12 = diff(Ee,eps,eps).subs(eps==0) == E2
s13 = diff(Pe,eps).subs(eps==0) == P1
s14 = diff(Pe,eps,eps).subs(eps==0) == P2

s15 = m == m0 + m2*(3*cos(th)^2-1)/2
s16 = k == k0 + k2*(3*cos(th)^2-1)/2
s17 = h == h0 + h2*(3*cos(th)^2-1)/2

sustituciones = (s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17)

In [30]:
dTdd = M.tensor_field(0,2,'dTdd')
ddTdd = M.tensor_field(0,2,'ddTdd')
for i in range(0,n+1):
    for j in range(0,n+1):
        dTdd[i,j] = diff(Tedd[i,j].expr(),eps)
        ddTdd[i,j] = diff(Tedd[i,j].expr(),eps,eps)
dTdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                              #d/de(T_\mu\nu)|e=0
ddTdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                         #d^2/de^2(T_\mu\nu)|e=0

In [31]:
# Calculamos ahora la primera derivada de la ecuación

show(deins[1,1]," = ", kappa*dTdd[1,1])
show(deins[2,2]," = ", kappa*dTdd[2,2])
show(deins[3,3]," = ", kappa*dTdd[3,3])

P1 = solve(deins[1,1].expr() == kappa*dTdd[1,1].expr(),P1)[0].rhs()

show("De aquí sacamos P1 -> P1(r) = ", P1)

0 ' = ' 8*pi*P1(r)*e^lamb(r)

0 ' = ' 8*pi*r^2*P1(r)

0 ' = ' 8*pi*r^2*P1(r)*sin(th)^2

'De aquí sacamos P1 -> P1(r) = ' 0

In [32]:
show(deins[0,0]," = ", kappa*dTdd[0,0])

E1 = solve(deins[0,0].expr() == kappa*dTdd[0,0].expr(),E1)[0].rhs()
E1 = E1.subs(sustituciones)
show("De aquí sacamos E1 -> E1(r) = ", E1)

0 ' = ' 8*pi*(2*E(r)*e^(3/2*nu(r))*u10(r) + 2*P(r)*e^(3/2*nu(r))*u10(r) + E1(r)*e^nu(r))

'De aquí sacamos E1 -> E1(r) = ' 0

In [33]:
show(deins[0,3].expr() == kappa*dTdd[0,3].expr())                                                           #Eq 48

-1.0*r*e^(-lamb(r))*omega(r, th)*sin(th)^2*diff(nu(r), r) == -8*pi*(r^2*E(r)*Omega1(eps) + r^2*Omega1(eps)*P(r) + r^2*E(r)*omega(r, th))*sin(th)^2

In [34]:
# Para la segunda derivada tenemos

show(ddeins[0,0].expr()," = ", kappa*ddTdd[0,0].expr())

-2*(r^2*e^(lamb(r) + 2*nu(r))*h(r, th) + 2*r^2*e^(lamb(r) + 2*nu(r))*k(r, th) + r^2*e^(lamb(r) + 2*nu(r))*m(r, th))*(4.0*r*e^(3*nu(r))*diff(lamb(r), r) + 4*(e^lamb(r) - 1)*e^(3*nu(r)))*e^(-2*lamb(r) - 4*nu(r))/r^4 + 1/4*(8*r^2*cos(th)*e^(lamb(r) + 2*nu(r))*omega(r, th)*sin(th)^2*diff(omega(r, th), th) - (4.0*r^3*e^(2*nu(r))*omega(r, th)^2*diff(lamb(r), r) + 8.0*r^3*e^(2*nu(r))*omega(r, th)^2*diff(nu(r), r) - 8.0*r^3*e^(2*nu(r))*omega(r, th)*diff(omega(r, th), r) + (16*r^2*e^lamb(r) - 16.0*r^2)*e^(2*nu(r))*omega(r, th)^2)*sin(th)^3 + 16*cos(th)*e^(lamb(r) + 3*nu(r))*diff(k(r, th), th) + (8.0*r^2*e^(3*nu(r))*h(r, th)*diff(nu(r), r)^2 + 8*r^2*e^(lamb(r) + 2*nu(r))*omega(r, th)^2 + 4.0*r^2*e^(3*nu(r))*diff(h(r, th), r)*diff(nu(r), r) + 8.0*r^2*e^(3*nu(r))*h(r, th)*diff(nu(r), r, r) + 32*(e^lamb(r) - 1)*e^(3*nu(r))*h(r, th) + (32*e^lamb(r) - 32.0)*e^(3*nu(r))*k(r, th) + 16*(2*e^lamb(r) - 1)*e^(3*nu(r))*m(r, th) + 16.0*r*e^(3*nu(r))*diff(k(r, th), r) - (-32.0*r*e^(3*nu(r))*h(r, th) - 48.0*r*e^(3*nu(r))*k(r, th) - 16.0*r*e^(3*nu(r))*m(r, th))*diff(lamb(r), r) - (4.0*r^2*e^(3*nu(r))*diff(nu(r), r) - 16.0*r*e^(3*nu(r)))*diff(m(r, th), r) - (-16.0*r*e^(3*nu(r))*h(r, th) + (4.0*r^2*e^(3*nu(r))*h(r, th) + 4.0*r^2*e^(3*nu(r))*m(r, th))*diff(lamb(r), r))*diff(nu(r), r))*sin(th))*e^(-lamb(r) - 2*nu(r))/(r^2*sin(th)) ' = ' 8*pi*(2*E(r)*e^(2*nu(r))*u10(r)^2 + 2*P(r)*e^(2*nu(r))*u10(r)^2 - 2*((2*r^2*E(r) + r^2*P(r))*omega(r, th)^2 + 2*(r^2*E(r)*Omega1(eps) + r^2*Omega1(eps)*P(r))*omega(r, th))*sin(th)^2 + 4*E1(r)*e^(3/2*nu(r))*u10(r) + 4*P1(r)*e^(3/2*nu(r))*u10(r) + 2*E(r)*e^(3/2*nu(r))*u20(r) + 2*P(r)*e^(3/2*nu(r))*u20(r) + E2(r)*e^nu(r) + 4*(2*E(r)*e^nu(r) + P(r)*e^nu(r))*h(r, th))

In [35]:
show(ddeins[1,1]," = ", kappa*ddTdd[1,1])

2*(r^2*e^(2*nu(r))*h(r, th) + 2*r^2*e^(2*nu(r))*k(r, th) + r^2*e^(2*nu(r))*m(r, th))*(-4.0*r*e^(2*nu(r))*d(nu)/dr + 4*(e^lamb(r) - 1)*e^(2*nu(r)))*e^(-4*nu(r))/r^4 - 1/4*(8*r^2*cos(th)*e^(lamb(r) + nu(r))*omega(r, th)*sin(th)^2*d(omega)/dth + (-4.0*r^3*e^nu(r)*omega(r, th)^2*d(nu)/dr - 8.0*r^3*e^nu(r)*omega(r, th)*d(omega)/dr - (16*r^2*e^lamb(r) + 8.0*r^2)*e^nu(r)*omega(r, th)^2)*sin(th)^3 + 16*cos(th)*e^(lamb(r) + 2*nu(r))*d(k)/dth + (8*r^2*e^(lamb(r) + nu(r))*omega(r, th)^2 - 4.0*r^2*e^(2*nu(r))*d(h)/dr*d(nu)/dr + 8.0*r^2*e^(2*nu(r))*m(r, th)*d^2(nu)/dr^2 + 32*(e^lamb(r) - 1)*e^(2*nu(r))*h(r, th) + (32*e^lamb(r) - 64.0)*e^(2*nu(r))*k(r, th) + 16*(2*e^lamb(r) - 1)*e^(2*nu(r))*m(r, th) - 16.0*r*e^(2*nu(r))*d(k)/dr + (-4.0*r^2*e^(2*nu(r))*h(r, th) + 4.0*r^2*e^(2*nu(r))*m(r, th))*d(nu)/dr^2 + (4.0*r^2*e^(2*nu(r))*d(nu)/dr + 16.0*r*e^(2*nu(r)))*d(m)/dr + (-16.0*r*e^(2*nu(r))*h(r, th) - 48.0*r*e^(2*nu(r))*k(r, th) - 16.0*r*e^(2*nu(r))*m(r, th))*d(nu)/dr)*sin(th))*e^(-2*nu(r))/(r^2*sin(th)) ' = ' 8*pi*(4*P(r)*e^lamb(r)*m(r, th) + P2(r)*e^lamb(r))

In [36]:
show(ddeins[2,2]," = ", kappa*ddTdd[2,2])

-2*(1.0*r^2*e^(2*nu(r))*(d(nu)/dr)^2 + 2.0*r^2*e^(2*nu(r))*d^2(nu)/dr^2 - 2.0*r*e^(2*nu(r))*d(lamb)/dr + (-1.0*r^2*e^(2*nu(r))*d(lamb)/dr + 2.0*r*e^(2*nu(r)))*d(nu)/dr)*(e^(lamb(r) + 2*nu(r))*h(r, th) + 2*e^(lamb(r) + 2*nu(r))*k(r, th) + e^(lamb(r) + 2*nu(r))*m(r, th))*e^(-2*lamb(r) - 4*nu(r)) + 1/4*(8*r^2*cos(th)*e^(lamb(r) + nu(r))*omega(r, th)*sin(th)^3*d(omega)/dth + (4.0*r^3*e^nu(r)*omega(r, th)^2*d(lamb)/dr + 4.0*r^3*e^nu(r)*omega(r, th)^2*d(nu)/dr - 8.0*r^3*e^nu(r)*omega(r, th)*d(omega)/dr - (16*r^2*e^lamb(r) + 8.0*r^2)*e^nu(r)*omega(r, th)^2)*sin(th)^4 + (16*r^2*e^(lamb(r) + nu(r))*omega(r, th)^2 - 4.0*r^2*e^(2*nu(r))*d(h)/dr*d(nu)/dr - 4.0*r^2*e^(2*nu(r))*d(m)/dr*d(nu)/dr - 16.0*r*e^(2*nu(r))*d(k)/dr + (4.0*r^2*e^(2*nu(r))*h(r, th) + 16.0*r^2*e^(2*nu(r))*k(r, th) + 4.0*r^2*e^(2*nu(r))*m(r, th))*(d(nu)/dr)^2 - 32.0*e^(2*nu(r))*k(r, th) + (-16.0*r*e^(2*nu(r))*h(r, th) - 16.0*r*e^(2*nu(r))*k(r, th))*d(lamb)/dr + (8.0*r*e^(2*nu(r))*h(r, th) + 16.0*r*e^(2*nu(r))*k(r, th) + 8.0*r*e^(2*nu(r))*m(r, th) + (-4.0*r^2*e^(2*nu(r))*h(r, th) - 16.0*r^2*e^(2*nu(r))*k(r, th) - 4.0*r^2*e^(2*nu(r))*m(r, th))*d(lamb)/dr)*d(nu)/dr + (8.0*r^2*e^(2*nu(r))*h(r, th) + 32.0*r^2*e^(2*nu(r))*k(r, th) + 8.0*r^2*e^(2*nu(r))*m(r, th))*d^2(nu)/dr^2)*sin(th)^2 + 16*e^(lamb(r) + 2*nu(r))*k(r, th))*e^(-lamb(r) - 2*nu(r))/sin(th)^2 ' = ' 8*pi*(4*r^2*P(r)*k(r, th) + r^2*P2(r))

In [37]:
show(ddeins[3,3]," = ", kappa*ddTdd[3,3])

-2*(1.0*r^2*e^(2*nu(r))*(d(nu)/dr)^2 + 2.0*r^2*e^(2*nu(r))*d^2(nu)/dr^2 - 2.0*r*e^(2*nu(r))*d(lamb)/dr + (-1.0*r^2*e^(2*nu(r))*d(lamb)/dr + 2.0*r*e^(2*nu(r)))*d(nu)/dr)*(e^(lamb(r) + 2*nu(r))*h(r, th) + 2*e^(lamb(r) + 2*nu(r))*k(r, th) + e^(lamb(r) + 2*nu(r))*m(r, th))*e^(-2*lamb(r) - 4*nu(r))*sin(th)^2 - 1/4*(8*r^2*cos(th)*e^(lamb(r) + nu(r))*omega(r, th)*sin(th)^3*d(omega)/dth + (-4.0*r^3*e^nu(r)*omega(r, th)^2*d(lamb)/dr - 4.0*r^3*e^nu(r)*omega(r, th)^2*d(nu)/dr + 8.0*r^3*e^nu(r)*omega(r, th)*d(omega)/dr - (24*r^2*e^lamb(r) - 24.0*r^2)*e^nu(r)*omega(r, th)^2)*sin(th)^4 + (16*r^2*e^(lamb(r) + nu(r))*omega(r, th)^2 + 4.0*r^2*e^(2*nu(r))*d(h)/dr*d(nu)/dr + 4.0*r^2*e^(2*nu(r))*d(m)/dr*d(nu)/dr - (32*e^lamb(r) - 32.0)*e^(2*nu(r))*k(r, th) + 16.0*r*e^(2*nu(r))*d(k)/dr - (4.0*r^2*e^(2*nu(r))*h(r, th) + 16.0*r^2*e^(2*nu(r))*k(r, th) + 4.0*r^2*e^(2*nu(r))*m(r, th))*(d(nu)/dr)^2 - (-16.0*r*e^(2*nu(r))*h(r, th) - 16.0*r*e^(2*nu(r))*k(r, th))*d(lamb)/dr - (8.0*r*e^(2*nu(r))*h(r, th) + 16.0*r*e^(2*nu(r))*k(r, th) + 8.0*r*e^(2*nu(r))*m(r, th) + (-4.0*r^2*e^(2*nu(r))*h(r, th) - 16.0*r^2*e^(2*nu(r))*k(r, th) - 4.0*r^2*e^(2*nu(r))*m(r, th))*d(lamb)/dr)*d(nu)/dr - (8.0*r^2*e^(2*nu(r))*h(r, th) + 32.0*r^2*e^(2*nu(r))*k(r, th) + 8.0*r^2*e^(2*nu(r))*m(r, th))*d^2(nu)/dr^2)*sin(th)^2 + 16*e^(lamb(r) + 2*nu(r))*k(r, th))*e^(-lamb(r) - 2*nu(r)) ' = ' 8*pi*(2*(r^4*E(r)*Omega1(eps)^2*e^(-nu(r)) + r^4*Omega1(eps)^2*P(r)*e^(-nu(r)) + (r^4*E(r)*e^(-nu(r)) + r^4*P(r)*e^(-nu(r)))*omega(r, th)^2 + 2*(r^4*E(r)*Omega1(eps)*e^(-nu(r)) + r^4*Omega1(eps)*P(r)*e^(-nu(r)))*omega(r, th))*sin(th)^4 + (4*r^2*P(r)*k(r, th) + r^2*P2(r))*sin(th)^2)

In [38]:
show(ddeins[0,3]," = ", kappa*ddTdd[0,3])

0 ' = ' -8*pi*(4*r^2*E(r)*Omega1(eps)*e^(1/2*nu(r))*u10(r) + 4*r^2*Omega1(eps)*P(r)*e^(1/2*nu(r))*u10(r) + 2*r^2*E1(r)*Omega1(eps) + r^2*E(r)*Omega2(eps) + r^2*Omega2(eps)*P(r) + 2*r^2*Omega1(eps)*P1(r) + 2*(2*r^2*E(r)*e^(1/2*nu(r))*u10(r) + 2*r^2*P(r)*e^(1/2*nu(r))*u10(r) + r^2*E1(r))*omega(r, th))*sin(th)^2

In [39]:
show(ddeins[1,2]," = ", kappa*ddTdd[1,2])

(4.0*r^2*e^nu(r)*omega(r, th)*sin(th)^2*d(omega)/dth + 2.0*r*e^(2*nu(r))*d(lamb)/dr*d(m)/dth + 2.0*r*e^(2*nu(r))*d(h)/dth*d(nu)/dr + (1.0*r^3*e^nu(r)*omega(r, th)^2*d(nu)/dr + 6.0*r^2*e^nu(r)*omega(r, th)^2)*cos(th)*sin(th) + 4.0*e^(2*nu(r))*d(k)/dth)*e^(-2*nu(r))/r ' = ' 0

In [40]:
# Aplicamos barotropic EoS
# Utilizando (6.11) de Reina con P1=0 y E1=0
E2*diff(P,r) - P2*diff(E,r) == 0

-P2(r)*diff(E(r), r) + E2(r)*diff(P(r), r) == 0

In [41]:
h

1/2*(3*cos(th)^2 - 1)*h2(r, th) + h0(r, th)

In [42]:
ddeins[0,0]

-2*(r^2*e^(lamb(r) + 2*nu(r))*h(r, th) + 2*r^2*e^(lamb(r) + 2*nu(r))*k(r, th) + r^2*e^(lamb(r) + 2*nu(r))*m(r, th))*(4.0*r*e^(3*nu(r))*d(lamb)/dr + 4*(e^lamb(r) - 1)*e^(3*nu(r)))*e^(-2*lamb(r) - 4*nu(r))/r^4 + 1/4*(8*r^2*cos(th)*e^(lamb(r) + 2*nu(r))*omega(r, th)*sin(th)^2*d(omega)/dth - (4.0*r^3*e^(2*nu(r))*omega(r, th)^2*d(lamb)/dr + 8.0*r^3*e^(2*nu(r))*omega(r, th)^2*d(nu)/dr - 8.0*r^3*e^(2*nu(r))*omega(r, th)*d(omega)/dr + (16*r^2*e^lamb(r) - 16.0*r^2)*e^(2*nu(r))*omega(r, th)^2)*sin(th)^3 + 16*cos(th)*e^(lamb(r) + 3*nu(r))*d(k)/dth + (8.0*r^2*e^(3*nu(r))*h(r, th)*d(nu)/dr^2 + 8*r^2*e^(lamb(r) + 2*nu(r))*omega(r, th)^2 + 4.0*r^2*e^(3*nu(r))*d(h)/dr*d(nu)/dr + 8.0*r^2*e^(3*nu(r))*h(r, th)*d^2(nu)/dr^2 + 32*(e^lamb(r) - 1)*e^(3*nu(r))*h(r, th) + (32*e^lamb(r) - 32.0)*e^(3*nu(r))*k(r, th) + 16*(2*e^lamb(r) - 1)*e^(3*nu(r))*m(r, th) + 16.0*r*e^(3*nu(r))*d(k)/dr - (-32.0*r*e^(3*nu(r))*h(r, th) - 48.0*r*e^(3*nu(r))*k(r, th) - 16.0*r*e^(3*nu(r))*m(r, th))*d(lamb)/dr - (4.0*r^2*e^(3*nu(r))

In [48]:
h0 = function('h0')(r,th)
k0 = function('k0')(r,th)
m0 = function('m0')(r,th)

h2 = function('h2')(r,th)
k2 = function('k2')(r,th)
m2 = function('m2')(r,th)

m = m0 + m2*(3*cos(th)^2-1)/2
k = k0 + k2*(3*cos(th)^2-1)/2
h = h0 + h2*(3*cos(th)^2-1)/2

In [54]:
ddeins.apply_map(lambda cmp: cmp.subs(h==h0+h2*(3*cos(th)^2-1)/2))
ddeins.apply_map(lambda cmp: cmp.subs(k==k0+k2*(3*cos(th)^2-1)/2))
ddeins.apply_map(lambda cmp: cmp.subs(m==m0+m2*(3*cos(th)^2-1)/2))

In [57]:
ddeins[0,0]

-2*(r^2*e^(lamb(r) + 2*nu(r))*h(r, th) + 2*r^2*e^(lamb(r) + 2*nu(r))*k(r, th) + r^2*e^(lamb(r) + 2*nu(r))*m(r, th))*(4.0*r*e^(3*nu(r))*d(lamb)/dr + 4*(e^lamb(r) - 1)*e^(3*nu(r)))*e^(-2*lamb(r) - 4*nu(r))/r^4 + 1/4*(8*r^2*cos(th)*e^(lamb(r) + 2*nu(r))*omega(r, th)*sin(th)^2*d(omega)/dth - (4.0*r^3*e^(2*nu(r))*omega(r, th)^2*d(lamb)/dr + 8.0*r^3*e^(2*nu(r))*omega(r, th)^2*d(nu)/dr - 8.0*r^3*e^(2*nu(r))*omega(r, th)*d(omega)/dr + (16*r^2*e^lamb(r) - 16.0*r^2)*e^(2*nu(r))*omega(r, th)^2)*sin(th)^3 + 16*cos(th)*e^(lamb(r) + 3*nu(r))*d(k)/dth + (8.0*r^2*e^(3*nu(r))*h(r, th)*d(nu)/dr^2 + 8*r^2*e^(lamb(r) + 2*nu(r))*omega(r, th)^2 + 4.0*r^2*e^(3*nu(r))*d(h)/dr*d(nu)/dr + 8.0*r^2*e^(3*nu(r))*h(r, th)*d^2(nu)/dr^2 + 32*(e^lamb(r) - 1)*e^(3*nu(r))*h(r, th) + (32*e^lamb(r) - 32.0)*e^(3*nu(r))*k(r, th) + 16*(2*e^lamb(r) - 1)*e^(3*nu(r))*m(r, th) + 16.0*r*e^(3*nu(r))*d(k)/dr - (-32.0*r*e^(3*nu(r))*h(r, th) - 48.0*r*e^(3*nu(r))*k(r, th) - 16.0*r*e^(3*nu(r))*m(r, th))*d(lamb)/dr - (4.0*r^2*e^(3*nu(r))